In [ ]:
!pip3 install facebook_business

In [2]:
import facebook_business as fb
print (fb.__version__)

16.0.0


In [ ]:
import sys
import os
import pandas as pd
import time
from datetime import datetime
from datetime import date
from datetime import timedelta
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.page import Page
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adreportrun import AdReportRun
from loguru import logger

from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
marker_token='XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
app_secret = 'XXXXXXXXXXXXXXXXX'
app_id = 'XXXXXXXXXXXXXXXXXXXXX'

# FROM FB to BIGQUERY

## Connect to BigQuery

In [ ]:
big_query_key_path = "XXXXXXX.json"
project_id = 'my_project'
dataset="test1"

def get_client_and_table(dataset):
    key_path = big_query_key_path
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

    client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
    return client, credentials

client, credentials = get_client_and_table(dataset=dataset)

In [ ]:
def save_to_bq(df_adset_list, table_name, schema, write_disp=None):
    table_id=f'{project_id}.{dataset}.{table_name}'
    if write_disp=="WRITE_TRUNCATE"
        job_config = bigquery.LoadJobConfig(
            schema=schema,
            write_disposition=write_disp)  
    else:
        job_config = bigquery.LoadJobConfig(
            schema=schema)          
    job = client.load_table_from_dataframe(df_adset_list, table_id, job_config=job_config) 
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))

## FB Adset to BigQuery

In [ ]:
def get_ads_set(ad_account):
    # Get AdSet By ad account
    # detail description of fields: https://developers.facebook.com/docs/marketing-api/reference/adgroup/
    field_list=[Ad.Field.name,Ad.Field.id,Ad.Field.account_id,Ad.Field.adset,Ad.Field.campaign_id,
                Ad.Field.configured_status, #The configured status of the ad. Use status instead of this field
                Ad.Field.effective_status,
                Ad.Field.status,
                Ad.Field.created_time, Ad.Field.updated_time,
                Ad.Field.source_ad,    
               ]
    ad_set_list=[]
    adsets = ad_account.get_ad_sets(fields=field_list)
    for adset in adsets:
        ad_set_list.append(adset)
    logger.info("count ad_sets: ", len(ad_set_list))
    return ad_set_list

In [ ]:
def create_df(ad_set_list):
    df_adset_list=pd.DataFrame(ad_set_list)
    df_adset_list['created_time']=pd.to_datetime(df_adset_list['created_time'])
    df_adset_list['updated_time']=pd.to_datetime(df_adset_list['updated_time'])
    return df_adset_list

In [ ]:
schema_adset=[
    bigquery.SchemaField('account_id', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('campaign_id', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('configured_status', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('created_time', 'DATETIME', 'NULLABLE'),
    bigquery.SchemaField('effective_status', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('id', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('name', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('status', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('updated_time', 'DATETIME', 'NULLABLE'),
]

In [ ]:
ad_set_list=get_ads_set(ad_account)
df_adset_list=create_df(ad_set_list)
_ = save_to_bq(df_adset_list, "adset_list", schema_adset, write_disp="WRITE_TRUNCATE"  )

## FB Ad to BigQuery

In [ ]:
def create_fb_account(ad_account_id):
    FacebookAdsApi.init(access_token=marker_token, api_version='v14.0')
    ad_account = AdAccount(ad_account_id)
    return ad_account

# count lids
def actions_ex(track):
    doc1=0
    if track is not None:
        for item in track:
            if item.get('action_type')=='offsite_conversion.fb_pixel_lead':
                doc1=int(item.get('value'))
    return doc1

# hourly range to end hour of range 
def hourly_int(h_range):
    doc1=0
    if h_range != 'unknown':
        doc1=int(h_range.split(' - ')[1].split(':')[0])+1
    return doc1   

# change type to int.
def to_int(h):
    return int(h if h is not None else 0) 

# change type to float.
def to_float(h):
    return float(h if h is not None else 0)  

# prepare one row for one hour
def adins(ad):

    doc={
        'account_id': ad.get('account_id'),
        'campaign_id': ad.get('campaign_id'),
        'adset_id': ad.get('adset_id'),
        'ad_id': ad.get('ad_id'),
        'impressions': to_int(ad.get('impressions')),
        'clicks': to_int(ad.get('clicks')),
        'cpc': to_float(ad.get('cpc')),
        'cpm': to_float(ad.get('cpm')),
        'cpp': to_float(ad.get('cpp')),
        'ctr': to_float(ad.get('ctr')),
        'inline_link_clicks': to_int(ad.get('inline_link_clicks')),
        'spend': to_float(ad.get('spend')),
        'reach': to_int(ad.get('reach')),
        'date_start': ad.get('date_start'),
        'date_stop': ad.get('date_stop'),
        'hourly_agg_by_audience_time_zone': hourly_int(ad.get('hourly_stats_aggregated_by_audience_time_zone')),
        'fb_pixel_lead': actions_ex(ad.get('actions')),
       
    }
    return doc

def req_fb(time_range_since,time_range_until,ad_account):
    # query FB to get ads stat
    params = {
        "time_increment": 1, # per day
        "level": "ad",
        'time_range': {'since': time_range_since, 'until': time_range_until}, 
        "breakdowns": "hourly_stats_aggregated_by_audience_time_zone", 
    }

    fields = [
        'ad_id',
        'adset_id',
        'campaign_id',
        'impressions',  
        'clicks',
        'cpc', # avg click cost
        'cpm', # avg 1000view cost
        'cpp', # avg 1000user cost 
        'ctr', 
        'inline_link_clicks',
        'reach',
        'spend',
        'actions'
    ]

    async_job = ad_account.get_insights_async(params=params,fields=fields)
    async_job.api_get()
    while async_job[AdReportRun.Field.async_status]!= 'Job Completed':
         time.sleep(1)
         async_job.api_get()
    time.sleep(1)
    result_cursor=async_job.get_result(params={"limit": 10000})
    list_ins=list(result_cursor)

    adins_list=[]
    if list_ins==[]:
        logger.info('Not stat')
        return pd.DataFrame()
    else:
        for item in list_ins:
            doc=adins(item)
            adins_list.append(doc)


    # check
    logger.info('request_fb_api - get list_ins - len list_ins: {}'.format(len(list_ins)))
    logger.success('request_fb_api - get list_ins - len adins_list: {}'.format(len(adins_list)))

    # to dataframe
    if len(adins_list)>0:
        df_adinsights=pd.DataFrame(adins_list)
        df_adinsights['date_start']=pd.to_datetime(df_adinsights['date_start'])
        df_adinsights['date_stop']=pd.to_datetime(df_adinsights['date_stop'])
    else:
        df_adinsights=pd.DataFrame()
    return df_adinsights,'OK'

In [ ]:
schema_ad=[
    bigquery.SchemaField('account_id', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('campaign_id', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('adset_id', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('ad_id', 'STRING', 'NULLABLE'),
    bigquery.SchemaField('date_start', 'DATETIME', 'NULLABLE'),
    bigquery.SchemaField('date_stop', 'DATETIME', 'NULLABLE'),
    bigquery.SchemaField('hourly_agg_by_audience_time_zone', 'INTEGER', 'NULLABLE'),
    bigquery.SchemaField('impressions', 'INTEGER', 'NULLABLE'),
    bigquery.SchemaField('clicks', 'INTEGER', 'NULLABLE'),
    bigquery.SchemaField('inline_link_clicks', 'INTEGER', 'NULLABLE'),
    bigquery.SchemaField('reach', 'INTEGER', 'NULLABLE'),
    bigquery.SchemaField('fb_pixel_lead', 'INTEGER', 'NULLABLE'),
    bigquery.SchemaField('cpc', 'FLOAT', 'NULLABLE'),
    bigquery.SchemaField('cpm', 'FLOAT', 'NULLABLE'),
    bigquery.SchemaField('cpp', 'FLOAT', 'NULLABLE'),
    bigquery.SchemaField('ctr', 'FLOAT', 'NULLABLE'),
    bigquery.SchemaField('spend', 'FLOAT', 'NULLABLE'),
]

In [ ]:
ad_account=create_fb_account(ad_account_id)
df_adinsights,err=req_fb(time_range_since,time_range_until,ad_account)
if err=='OK':
    _ = save_to_bq(df_adinsights, "ad_insights", schema_ad)

# OTHER Methods

## breakdowns:
list<enum{ad_format_asset, age, app_id, body_asset, call_to_action_asset, страна, description_asset, пол, image_asset, Impression_Device, link_url_asset, product_id, region, skan_conversion_id, title_asset, video_asset, dma, Frequency_value, hourly_stats_aggregated_by_advertiser_time_zone, hourly_stats_aggregated_by_advertiser_time_zone, hourly_form_stats_aggregated_by_placeer_time_zone, publisher_pageid_position_time_zone, place_audience_time_zone, платформа_устройства}>

- https://developers.facebook.com/docs/marketing-api/insights/breakdowns
- https://developers.facebook.com/docs/marketing-api/insights/breakdowns#combiningbreakdowns

## ad_account.get_insights - info about account

In [ ]:
fields = [
    'clicks'
]
params = {
    'time_range': {'since':'2022-01-19','until':'2022-02-18'},
    'filtering': [],
    'level': 'account',
    'breakdowns': ['age'],
}
result=(ad_account.get_insights(
    fields=fields,
    params=params,
))

In [ ]:
fields = [
    'clicks'
]
params = {
    'time_range': {'since':'2022-01-19','until':'2022-02-18'},
    'filtering': [],
    'level': 'account',
    'breakdowns': ['gender','age'],
}
result=(ad_account.get_insights(
    fields=fields,
    params=params,
))

## ad_account.get_ad_sets - info about ads group

In [ ]:
adsets = ad_account.get_ad_sets(fields=[AdSet.Field.name])
for adset in adsets:
    print(adset[AdSet.Field.name])

## ad_account.get_ads - info about ads

In [ ]:
ad_iter = ad_account.get_ads(fields=[Ad.Field.name])
print(ad_iter[24])
print(ad_iter[1][Ad.Field.name])

## campaign.get_ads

campaign = Campaign(campaign_id)
ad_iter = campaign.get_ads(fields=[Ad.Field.name])

In [ ]:
campaign_id = '2323523525235235325'
campaign = Campaign(campaign_id)
ad_iter = campaign.get_ads(fields=[Ad.Field.name])
for ad in ad_iter:
    print(ad[Ad.Field.name]) 

In [ ]:
fields = [
    'impressions',
    'clicks',
]
params = {
    'date_preset': 'this_month',
    'breakdowns': 'age',
}
print(campaign.get_insights(
  fields=fields,
  params=params,
))

## ad_set.get_ads

ad_set = AdSet(adset_id)
ad_iter = ad_set.get_ads(fields=[Ad.Field.name])
ad_set.get_insights - получить статистику по группе объявлений

In [ ]:
adset_id = '345345346346346346'
ad_set = AdSet(adset_id)
field_list=[Ad.Field.name,Ad.Field.id,Ad.Field.account_id,Ad.Field.ad_review_feedback,Ad.Field.adlabels,
          Ad.Field.adset,Ad.Field.bid_amount,Ad.Field.campaign,Ad.Field.campaign_id,
          Ad.Field.configured_status,Ad.Field.effective_status,Ad.Field.status,
          Ad.Field.conversion_domain,Ad.Field.created_time,
          Ad.Field.preview_shareable_link,Ad.Field.source_ad, Ad.Field.tracking_specs,Ad.Field.updated_time]
ad_iter = ad_set.get_ads(fields=field_list)
for ad in ad_iter:
    print(ad[Ad.Field.name])

In [ ]:
fields = [
    'impressions',
    'clicks',
]
params = {
    'date_preset': 'this_month',
    'breakdowns': 'age',
}
print(ad_set.get_insights(
  fields=fields,
  params=params,
))

## ad.api_get

In [ ]:
def each_ad(adid):
    ad_id = adid
    ad = Ad(ad_id)
    print(ad_id)
    field_list=[Ad.Field.name,Ad.Field.id,Ad.Field.account_id,Ad.Field.ad_review_feedback,Ad.Field.adlabels,
          Ad.Field.adset,Ad.Field.bid_amount,Ad.Field.campaign,Ad.Field.campaign_id,
          Ad.Field.configured_status,Ad.Field.effective_status,Ad.Field.status,
          Ad.Field.conversion_domain,Ad.Field.created_time,
          Ad.Field.preview_shareable_link,Ad.Field.source_ad, Ad.Field.tracking_specs,Ad.Field.updated_time]
    print(ad.api_get(fields=field_list))
    print('-----------------------------')
    
    fields = [
        'ad_id',
        'adset_id',
        'campaign_id',
        'impressions',  
        'clicks',
        'conversions',  
        'cpc', 
    ]
    
    params = {
        'date_preset': 'this_month',
        'breakdowns': ['age'],
    }
    print(ad.get_insights(
      fields=fields,
      params=params,
    ))
    print('=======================================================')

In [ ]:
ad_iter = ad_account.get_ads(fields=[Ad.Field.name,Ad.Field.id])
for i in range(1,3):
    each_ad(ad_iter[1][Ad.Field.id])